## 1. 필요한 라이브러리 불러오기

In [2]:
from pathlib import Path
import pandas as pd
import sys

ROOT_DIR = Path.cwd().parent.parent
sys.path.append(ROOT_DIR)

## 2. 데이터 준비하기


### 2.1 원본 데이터 읽어오기

In [3]:
df1 = pd.read_json(
    ROOT_DIR / "datasets/민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.json"
)
df2 = pd.read_json(
    ROOT_DIR / "datasets/민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Validation.json"
)
df1.shape, df2.shape

FileNotFoundError: File /Users/sdh/Dev/02_production_projects/humetro-ai-assistant/datasets/민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.json does not exist

### 2.2 데이터프레임 합치기

In [4]:
# 전체 데이터셋중 개체명에 "도시철도, 지하철" 이 있는 경우만 추출함.
merged = pd.concat([df1, df2], ignore_index=True)
merged.shape

NameError: name 'df1' is not defined

### 2.3도시철도 관련 데이터만 추출하기

In [57]:
merged["용어사전"].value_counts()
keywords = ["자하철", "도시철도", "호선"]

# '용어사전' 컬럼에 키워드가 하나라도 포함된 행만 추출
subway_df = merged[
    merged["용어사전"].apply(lambda x: any(kw in str(x) for kw in keywords))
]

print(subway_df.shape)

(471, 15)


### 2.4 도시철도 관련 질문 모두 수집하기

In [59]:
customer_questions = list(subway_df["고객질문(요청)"].unique())
for q in customer_questions:
    print(q)
    print("-" * 100)
print(len(customer_questions))

1호선 첫 차가 몇시 운영이에요?
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
지하철도 통제가 되나요?
----------------------------------------------------------------------------------------------------
그럼 9호선은 아예 자전거 가지고 타는 게 불가능한가요?
----------------------------------------------------------------------------------------------------
몇호선 타야해요?
----------------------------------------------------------------------------------------------------
고속터미널은 3호선 아니었나요?
----------------------------------------------------------------------------------------------------
1호선 급행도 노량진에서 하차하나요?
----------------------------------------------------------------------------------------------------
9호선도 급행이 있나요?
----------------------------------------------------------------------------------------------------
9호선은 얼마나 타야 종합운동장 나오나요?
---------------------------------------

### 2.5 LLM으로 질문 전처리하기
  - 컨텍스트 길이를 고려해서 절반씩 나눠서 LLM 호출
  - 단순한 요약 및 재작성 태스크이므로 모델은 비교적 작은 gpt-4o-mini 사용
  - 답변은 json 형식으로 반환

In [74]:
import openai


def preprocess_question(question):
    messages = [
        {
            "role": "system",
            "content": "당신은 대중교통 관련 질문을 전처리하는 도우미입니다.",
        },
        {
            "role": "user",
            "content": f"""다음은 대중교통 관련 민원인의 질문 85개입니다.
            이 질문들을 참고해서 다음의 작업을 수행하세요
            1. 중복된 질문은 하나로 줄이고, 중복되지 않는 질문은 그대로 두세요.
            2. 질문을 정확하고 구체적으로 다시 작성하세요.
            3. 일반적인 대중교통 질문이 아닌 경우 제외하세요.
            4. 이외에도 맥락상 필요하다고 생각하는 질문이 있는 경우 추가하세요.
            5. 최종적으로 전처리된 질문을 반환하세요.
            답변은 json 형식으로 반환하세요.
            ---
            질문:
            {question}
            """,
        },
    ]

    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            response_format={"type": "json_object"},
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        # 응답 형식 옵션을 제거하고 재시도
        try:
            response = openai.chat.completions.create(
                model="gpt-4o-mini", messages=messages
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"두 번째 시도 중 오류 발생: {e}")
            return None


q1 = preprocess_question("\n".join(customer_questions[:40]))
q2 = preprocess_question("\n".join(customer_questions[40:]))

### 2.6 전처리 결과 합치고 평가하기
  - 전처리 결과 현재의 RAG 데이터로 답변할 수 없는 질문이 대부분(길찾기 등)
  - 이 질문 세트는 제외하기로 결정함

In [76]:
import json

q1_dict = json.loads(q1)
q2_dict = json.loads(q2)

questions = q1_dict["questions"] + q2_dict["questions"]
print(len(questions))
questions

64


[{'question': '1호선의 첫 차 운영 시간은 몇 시인가요?'},
 {'question': '지하철도 통제가 가능한가요?'},
 {'question': '9호선에서 자전거를 가지고 타는 것이 불가능한가요?'},
 {'question': '강남으로 가기 위해서 어떤 호선을 이용해야 하나요?'},
 {'question': '고속터미널역은 3호선에 위치한 게 맞나요?'},
 {'question': '1호선 급행열차는 노량진역에서 하차하나요?'},
 {'question': '9호선에도 급행열차가 존재하나요?'},
 {'question': '9호선에서 종합운동장역까지는 얼마나 걸리나요?'},
 {'question': '오산시청역에서 강남으로 가는 버스는 몇 번인가요?'},
 {'question': '시청에서 강남까지 이동하는 데 소요되는 시간은 얼마나 되나요?'},
 {'question': '신풍역에서 동작역까지 가는 경로를 알려줄 수 있나요?'},
 {'question': '세종문화회관이 종점인가요?'},
 {'question': '동대문역사는 2호선으로 가나요?'},
 {'question': '4호선에서 가장 많은 승객이 있는 역은 어디인가요?'},
 {'question': '4호선에는 승객이 많은 급행역이 있나요?'},
 {'question': '2호선과 신분당선의 막차 시간은 연결되어 있나요?'},
 {'question': '수서역에서 삼성병원으로 가는 버스는 있나요? 아니면 지하철을 이용해야 하나요?'},
 {'question': '4호선은 어떤 방향으로 승차해야 하나요?'},
 {'question': '롯데월드를 이용할 때 지하철 할인 요금이 있나요?'},
 {'question': '충무로역은 몇 호선에 위치해 있나요?'},
 {'question': '대전에는 지하철 노선이 없나요?'},
 {'question': '환승할 때 1호선에서 2호선으로 바꿔 타야 하나요?'},
 {'question': '선정릉역과 등촌역이 둘 다 9호선인가요?'},
 {'

## 3. 도큐먼트 데이터 전처리 및 적재

1. 부산교통공사 홈페이지의 "이용안내" 섹션

    <img src="/Users/sdh/Dev/projects/humetro-ai-assistant/assets/부산교통공사_이용안내.png" width="50%" alt="대체텍스트">


2. 부산교통공사 직원 교육용 교재
 - "일타역무 교재"
3. 부산교통공사 역무정보 편집본
 - "역무 위키" 

### 3.1 마크다운 형식으로 저장된 데이터의 노이즈 제거(with gpt-4o)

In [86]:
PROMPT_TEMPLATE = """
당신은 웹에서 크롤링된 데이터가 포함된 마크다운 파일을 처리하는 AI입니다. 주어진 마크다운 파일에는 다양한 정보가 섞여 있으며, 이 중에는 불필요하거나 중복되는 내용이 포함되어 있을 수 있습니다.

**당신의 임무:**

1.  **입력 분석:** 아래 제공될 마크다운 형식의 텍스트(`[입력 마크다운 내용]`)를 분석합니다.
2.  **핵심 정보 추출:** 텍스트 내용 중에서 의미 있고 중요한 정보만을 선별하여 추출합니다. 불필요한 정보(예: 광고, 사이트 메뉴, 관련 없는 링크, 서식 오류, 머리말/꼬리말 등)는 제거합니다.
3.  **콘텐츠 재구성 및 마크다운 생성:** 추출된 핵심 정보들을 논리적으로 재구성하고 정리하여, 새로운 마크다운 형식의 본문을 생성합니다. 내용은 명확하고 간결하게 작성되어야 합니다. 원본의 핵심 의미를 유지해야 합니다.
4.  **제목 생성:** 생성된 마크다운 본문의 내용을 가장 잘 요약하고 나타내는 적절하고 간결한 제목을 생성합니다.
5.  **JSON 출력:** 최종 결과를 반드시 다음 키를 포함하는 **유효한 JSON 형식**으로 출력합니다. 다른 부가적인 설명 없이 오직 JSON 객체만을 반환해야 합니다.
    * `title` (string): 4번 단계에서 생성한 제목
    * `content` (string): 3번 단계에서 생성한 마크다운 형식의 본문 내용

**처리할 입력 마크다운 내용:**

{markdown_content}

요청:
위의 지침에 따라 주어진 마크다운 내용을 처리하고, 최종 결과물을 JSON 형식으로만 출력해 주세요.
다른 부가적인 설명은 포함하지 않고 오직 유효한 JSON 객체만을 반환해 주세요.
만약 별다른 정보가 없는 경우 제목과 컨텐츠에 빈 문자열을 반환해 주세요.
"""


def preprocess_markdown(markdown_text: str) -> str:
    """
    마크다운 형식의 데이터를 전처리하는 함수
    """
    response = openai.chat.completions.create(
        model="gpt-4o",  # Using gpt-4o as specified in the original snippet
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant specialized in processing and summarizing web-crawled markdown content into a structured JSON format. Respond ONLY with the valid JSON object containing 'title' and 'content' keys, based on the user's instructions.",
            },
            {
                "role": "user",
                "content": PROMPT_TEMPLATE.format(markdown_content=markdown_text),
            },
        ],
        response_format={"type": "json_object"},  # Ensure JSON output
        temperature=0.2,  # Lower temperature for more focused and deterministic output
    )

    return json.loads(response.choices[0].message.content.strip())

#### 3.1.1 모든 노이즈가 있는 마크다운 파일에 대해서 실행

In [87]:
rag_docs_dir = ROOT_DIR / "datasets/Rag_Docs"
md_files = list(rag_docs_dir.glob("*.md"))


results = []
for md_file in md_files:
    with open(md_file, "r") as f:
        markdown_text = f.read()
    process_result = preprocess_markdown(markdown_text)
    print(process_result)
    results.append(process_result)

print(results)

{'title': '부산 도시철도 에스컬레이터 현황', 'content': '부산 도시철도 에스컬레이터는 총 652대가 운영 중입니다. 각 호선별 에스컬레이터 수는 다음과 같습니다:\n\n- **1호선**: 141대\n- **2호선**: 206대\n- **3호선**: 174대\n- **4호선**: 131대\n\n각 역별 에스컬레이터 수는 다음과 같습니다:\n\n### 1호선\n- 다대포해수욕장: 13대\n- 다대포항: 21대\n- 낫개: 13대\n- 신장림: 12대\n- 장림: 14대\n- 동매: 12대\n- 하단: 2대\n- 괴정: 2대\n- 대티: 3대\n- 서대신: 7대\n- 자갈치: 6대\n- 남포: 7대\n- 중앙: 2대\n- 부산: 6대\n- 범내골: 2대\n- 서면(1): 10대\n- 연산(1): 4대\n- 교대(1): 1대\n- 동래(1): 2대\n- 범어사: 1대\n- 노포: 1대\n\n### 2호선\n- 벡스코(시립미술관): 2대\n- 센텀시티: 14대\n- 민락: 8대\n- 수영(2): 6대\n- 광안: 8대\n- 금련산: 8대\n- 남천: 8대\n- 경성대·부경대: 8대\n- 대연: 8대\n- 못골: 8대\n- 지게골: 8대\n- 문현: 8대\n- 국제금융센터·부산은행: 4대\n- 전포: 4대\n- 서면(2): 8대\n- 부암: 8대\n- 가야: 8대\n- 동의대: 8대\n- 개금: 8대\n- 사상(2): 2대\n- 구남: 8대\n- 구명: 8대\n- 수정: 6대\n- 화명: 4대\n- 호포: 2대\n- 증산: 8대\n- 부산대양산캠퍼스: 4대\n- 남양산: 10대\n- 양산: 12대\n\n### 3호선\n- 수영(3): 4대\n- 망미: 12대\n- 배산: 16대\n- 물만골: 12대\n- 연산(3): 20대\n- 거제(3): 5대\n- 종합운동장: 13대\n- 사직: 3대\n- 미남(3): 4대\n- 만덕: 20대\n- 남산정: 5대\n- 숙등: 12대\n- 덕천(3): 10대\n- 구포: 9대\n- 강서구청: 12대\n- 체육공원: 

#### 3.1.2 실행결과를 마크다운으로 저장

In [89]:
results

[{'title': '부산 도시철도 에스컬레이터 현황',
  'content': '부산 도시철도 에스컬레이터는 총 652대가 운영 중입니다. 각 호선별 에스컬레이터 수는 다음과 같습니다:\n\n- **1호선**: 141대\n- **2호선**: 206대\n- **3호선**: 174대\n- **4호선**: 131대\n\n각 역별 에스컬레이터 수는 다음과 같습니다:\n\n### 1호선\n- 다대포해수욕장: 13대\n- 다대포항: 21대\n- 낫개: 13대\n- 신장림: 12대\n- 장림: 14대\n- 동매: 12대\n- 하단: 2대\n- 괴정: 2대\n- 대티: 3대\n- 서대신: 7대\n- 자갈치: 6대\n- 남포: 7대\n- 중앙: 2대\n- 부산: 6대\n- 범내골: 2대\n- 서면(1): 10대\n- 연산(1): 4대\n- 교대(1): 1대\n- 동래(1): 2대\n- 범어사: 1대\n- 노포: 1대\n\n### 2호선\n- 벡스코(시립미술관): 2대\n- 센텀시티: 14대\n- 민락: 8대\n- 수영(2): 6대\n- 광안: 8대\n- 금련산: 8대\n- 남천: 8대\n- 경성대·부경대: 8대\n- 대연: 8대\n- 못골: 8대\n- 지게골: 8대\n- 문현: 8대\n- 국제금융센터·부산은행: 4대\n- 전포: 4대\n- 서면(2): 8대\n- 부암: 8대\n- 가야: 8대\n- 동의대: 8대\n- 개금: 8대\n- 사상(2): 2대\n- 구남: 8대\n- 구명: 8대\n- 수정: 6대\n- 화명: 4대\n- 호포: 2대\n- 증산: 8대\n- 부산대양산캠퍼스: 4대\n- 남양산: 10대\n- 양산: 12대\n\n### 3호선\n- 수영(3): 4대\n- 망미: 12대\n- 배산: 16대\n- 물만골: 12대\n- 연산(3): 20대\n- 거제(3): 5대\n- 종합운동장: 13대\n- 사직: 3대\n- 미남(3): 4대\n- 만덕: 20대\n- 남산정: 5대\n- 숙등: 12대\n- 덕천(3): 10대\n- 구포: 9대\n- 강서구청: 12대\n- 체육공

In [90]:
for result in results:
    try:
        filepath = ROOT_DIR / "datasets/final_docs" / f"{result['title']}_processed.md"
        with open(filepath, "w") as f:
            f.write(result["content"])
    except Exception as e:
        print(f"Error writing to {filepath}: {e}")

Error writing to /Users/sdh/Dev/projects/humetro-ai-assistant/datasets/final_docs/_processed.md: 'title'


## 4. 도큐먼트를 바탕으로 Q/A 데이터 합성

## 5. 